In [1]:
pip install pytest

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ipytest

Note: you may need to restart the kernel to use updated packages.


In [3]:
import ipytest
ipytest.autoconfig()

In [4]:
# adding required packages
import findspark

In [5]:
#To make pyspark importable as a regular library
findspark.init()

In [6]:
#importing pyspark related package
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [7]:
%run /notebook/dataproduct/assesment_nyc_job_posting/lib/data_profiling_transform.ipynb

Data profile Function Imported


In [8]:
%run /notebook/dataproduct/assesment_nyc_job_posting/utils/spark_session.ipynb

Spark Session Imported


In [9]:
def get_spark_conf():
    # Configure Spark settings
    spark_conf = SparkConf()
    spark_conf.set("spark.executor.instances", "4") # 4 instance per node

    # Set the number of executor cores
    spark_conf.set("spark.executor.cores", "2")  # Use 2 cores per executor

    # Set the executor memory
    spark_conf.set("spark.executor.memory", "1g")  # Use 1GB memory per executor

    # Set the driver memory
    spark_conf.set("spark.driver.memory", "2g")    # Use 2GB memory for the driver
    
    return spark_conf

In [10]:
job_name = 'UnitTest'

#setting spark conf before creating spark session
spark_conf = get_spark_conf()
    
# Create a SparkSession with the configured settings
spark = get_spark_session(spark_conf, job_name)

In [11]:
%%ipytest

def test_calculate_missing_value_counts():
    # Create a Spark DataFrame for testing
    data = [
        (87990, "Internal", None),
        (87991, None, "1"),
        (87992, "External", "NULL"),
        (87993, "Internal", "M3"),
        (87994, "External", "M2")
    ]
    columns = ["Job ID", "Posting Type", "Level"]
    df = spark.createDataFrame(data, columns)

    # Call the function to be tested
    result_df = calculate_missing_value_counts(df)

    # Define the expected result DataFrame
    expected_data = [
        (0, 1, 2)
    ]
    expected_columns = ["Job ID_missing", "Posting Type_missing", "Level_missing"]
    expected_df = spark.createDataFrame(expected_data, expected_columns)

    # Compare the actual and expected DataFrames
    assert result_df.collect() == expected_df.collect()

.                                                                                            [100%]
1 passed in 19.06s


In [12]:
%%ipytest

# Test the calculate_summary_stats function
def test_calculate_summary_stats():
    
    data = [
        (1, 5, 10),
        (2, 10, 20),
        (3, 15, 30),
    ]
    columns = ["# Of Positions", "Salary Range From", "Salary Range To"]
    
    
    numerical_columns = ["# Of Positions", "Salary Range From", "Salary Range To"]
    
    test_data = spark.createDataFrame(data, columns)
    
    result = calculate_summary_stats(test_data, numerical_columns)
    
    # Ensure that the resulting DataFrame has the expected columns
    expected_columns = ["summary", "# Of Positions", "Salary Range From", "Salary Range To"]
    assert result.columns == expected_columns
    
    # Ensure that the "mean" value for "# Of Positions" matches the expected value
    assert result.filter(result["summary"] == "mean").select("# Of Positions").collect()[0]["# Of Positions"] == '2.0'
    
    # Ensure that the "max" value for "Salary Range From" matches the expected value
    assert result.filter(result["summary"] == "max").select("Salary Range From").collect()[0]["Salary Range From"] == '15'

.                                                                                            [100%]
1 passed in 3.71s


In [13]:
%%ipytest

# Test the is_categorical function
def test_is_categorical():
    
    data = [
        ("Internal", 2),
        ("External", 20),
        ("Internal", 3),
        ("External", 40),
        ("Internal", 5),
        ("External", 60),
    ]
    columns = ["Posting Type", "# Of Positions"]
    test_data = spark.createDataFrame(data, columns)
    
    
    # Test a categorical column with distinct count above threshold
    is_cat, distinct_count = is_categorical(test_data, "Posting Type",2)
    assert is_cat is True
    assert distinct_count == 2
    
    # Test a categorical column with distinct count below threshold
    is_cat, distinct_count = is_categorical(test_data, "# Of Positions",2)
    assert is_cat is False
    assert distinct_count == 6

.                                                                                            [100%]
1 passed in 12.34s


In [14]:
%%ipytest

# Test the is_categorical function
def test_is_categorical():
    
    data = [
        ("Internal", 2),
        ("External", 20),
        ("Internal", 3),
        ("External", 40),
        ("Internal", 5),
        ("External", 60),
        ("Internal", 32),
        ("External", 4),
        ("Internal", 53),
        ("External", 10),
        ("Internal", 37),
        ("External", 41),
        ("Internal", 52),
        ("External", 61)
    ]
    columns = ["Posting Type", "# Of Positions"]
    test_data = spark.createDataFrame(data, columns)
    
    
    # Test a categorical column with distinct count below threshold
    is_categorical_cond, distinct_values, top_values = profile_categorical_column(test_data, "Posting Type")
    assert is_categorical_cond == True
    assert distinct_values == 2
    assert top_values.collect() == spark.createDataFrame([('Internal',7),('External',7)],columns).collect()
    
    # Test a categorical column with distinct count above threshold
    is_categorical_cond, distinct_values, top_values  = profile_categorical_column(test_data, "# Of Positions")
    assert is_categorical_cond == False
    assert distinct_values == None
    assert top_values == None

.                                                                                            [100%]
1 passed in 9.72s
